In [1]:
import matplotlib.pyplot as plt
# from networkx import minimum_cut, DiGraph
from mip import *

from src.read_instance import MDOVRP

In [2]:
filepath = "Instancias/ES-n78-m2-Q10138.txt"
# filepath = "Instancias/Vrp-Set-A/A-n80-m2-Q60.vrp"

In [3]:
N, D, V, Q, q, c, coord_x, coord_y = MDOVRP(filepath)

Usando a formulação $MDOVRP_{2i− flv}$ de Lalla-Ruiz e Mes (2019)


    Lalla-Ruiz, Eduardo, and Martijn Mes. "Mathematical formulations and improvements for the multi-depot open vehicle routing problem." Optimization Letters 15 (2021): 271-286.

In [4]:
def model_MDOVRP_MO(N, D, V, Q, q, c, max_distance, num_vehicles, name, solver_name, distance_cost, demand_cost):
    # CONSTANTES
    # limite de carros total
    NUM_VEICULOS = num_vehicles

    # Funcao objetivo
    custo_distancia = distance_cost
    custo_demanda = demand_cost

    #cria o modelo
    model = Model(name, solver_name = solver_name)

    # Variaveis de decisao
    x = [[model.add_var(var_type=BINARY) if i!=j else model.add_var(lb=0, ub=0) for i in V] for j in V]
    u = [[model.add_var(var_type=CONTINUOUS) if i!=j else model.add_var(lb=0, ub=0) for i in V] for j in V]
    y = [model.add_var(var_type=BINARY) for i in N]
    w = [model.add_var(var_type=BINARY) for i in N]

    # -------------------------------------------------------
    model.objective = minimize(custo_distancia*xsum(x[i][j] * c[i,j] for i in V for j in V if i != j) - custo_demanda*xsum(q[j]*w[j] for j in N))

    # Restricao 2
    for j in N:
        model += xsum(x[i][j] for i in V if i != j) == w[j]

    # Restricao 18

    for j in N:
        model += (xsum(x[i][j] for i in V if i != j) - xsum(x[j][i] for i in N if i != j)) >= 0

    # Restricao 19
    for i in V:
        for j in V:
            if i != j:
                model += x[i][j] + x[j][i] <= 1

    # Restricao 20
    # Ninguem volta para o depósito
    model += xsum(x[j][k] for k in D for j in V) == 0

    # Restricao 21 modificada -> se cidade não é atendida, restrição é esquecida
    for j in N:
        model += (xsum(u[i][j] for i in V if i != j) - xsum(u[j][i] for i in V if i != j) - q[j]) >= -Q*(1 - w[j])

    # restrição que liga x e w
    for j in N:
        xsum(x[i][j] for i in V if i != j) == w[j]

    #Restricao 22
    for i in N:
        for j in N:
            model += (Q - q[i])*x[i][j] >= u[i][j]

    # Restricao 23
    for k in D:
        for j in N:
            model += Q * x[k][j] >= u[k][j]


    # # Restricao 8
    # menor distancia de algum deposito ate a cidade
    d = [min([c[j, i] for j in D]) for i in N]
    # menor distancia entre cidade i e quaisquer outra cidade
    r = [min([c[j, i] for j in N if i != j]) for i in N]
    # máxima menor distancia entre duas cidades
    M = max(r)

    # d[i] >= r[i] <=> é mais longe ir de algum depósito do q de outra cidade => y in {0, 1}
    # d[i] < r[i] <=> é mais perto sair de algum depósito do q de qualquer outra cidade => y = 1
    for i in N:
        model += d[i] + M * y[i] >= r[i]*w[i]

    # Restricao 9

    for i in N:
        k_l = D[np.argmin([c[j, i] for j in D], axis=0)]
        model += x[k_l][i] >= y[i]

    model += xsum(x[k][i] for k in D for i in N) <= NUM_VEICULOS

    # Restricao na demanda relacionado ao limite de caminhões
    model += xsum(q[j]*w[j] for j in N) <= NUM_VEICULOS*Q

    # Restricao de distancia maxima entre duas cidades

    for i in V:
        for j in V:
            if i != j:
                model += c[i,j]*x[i][j] <= max_distance
    return (model, x, u, y, w)

In [5]:
num_vehicles = 10
max_distance = 180

distance_cost = 1
demand_cost = 10

model, x, u, y, w = model_MDOVRP_MO(N, D, V, Q, q, c, max_distance, num_vehicles, 'v0', GUROBI, distance_cost, demand_cost)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID
Academic license - for non-commercial use only - registered to thiago.giachetto@aluno.ufop.edu.br


In [6]:
# Resolve o modelo        
# model.optimize(max_seconds=7200)
model.optimize(max_seconds=120)
print("status: {} objective value : {} best possible: {}".format(model.status, model.objective_value, model.objective_bound))

Set parameter TimeLimit to value 120
Set parameter NodeLimit to value 1073741824
Set parameter SolutionLimit to value 1073741824
Set parameter IntFeasTol to value 1e-06
Set parameter Method to value 3
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Academic license - for non-commercial use only - registered to thiago.giachetto@aluno.ufop.edu.br
Optimize a model with 18323 rows, 12320 columns and 59808 nonzeros
Model fingerprint: 0x691a671d
Variable types: 6162 continuous, 6158 integer (6158 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [3e+00, 5e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+05]
Found heuristic solution: objective 0.0000000
Presolve removed 13871 rows and 6537 columns
Presolve time: 0.19s
Presolved: 4452 rows, 5783 columns, 21573 nonzeros
Variable types: 2832 continuous, 2951 integer (2951 binary)
Found heuristic solution: ob

---

## Testando resultados

In [7]:
# dist percorrida
sum([(1 if x[i][j].x>0.5 else 0) * c[i,j] for i in V for j in V if i != j])

2457.7910000000006

In [9]:
# demanda total atendida
print(f"demanda atendida: {sum(q[j]*w[j].x for j in N)} | limite demanda: {num_vehicles*Q} | total demanda: {sum(q)}")

demanda atendida: 83751.0 | limite demanda: 101380 | total demanda: 83751


In [ ]:
# # --- PLOTA O GRAFICO ---
# plt.figure(figsize=(10, 6))
# plt.scatter(coord_x[0:], coord_y[0:])
# for i in N:
#     plt.annotate(f"{i}", (coord_x[i], coord_y[i]))
# for i in D:
#     plt.plot(coord_x[i], coord_y[i], c = 'r', marker = 's')

# for i in V:
#     for j in V:
#         if x[i][j].x > 0:
#             plt.plot([coord_x[i], coord_x[j]], [coord_y[i], coord_y[j]], c='g', zorder=0)
    
# #escala dos eixos
# plt.yticks([i for i in range(1,80,20)]); 
# plt.xticks([i for i in range(1,80,20)]); 

In [10]:
# para onde cada veículo vai
for k in D:
    print(f"{k} - ", end='')
    print([i for i, el in enumerate(u[k]) if el.x > 0])

76 - [6, 28]
77 - [8, 16, 17, 22, 32, 61, 62, 69]


In [11]:
# caminho dos carros olhando arcos
for k in D:
    print(f"Depot: {k}")
    for i, el in enumerate(x[k]):
        if el.x > 0:
            visitados = [i]
            next_list = [j for j, el in enumerate(x[i]) if el.x > 0]
            while len(next_list) > 0:
                if len(next_list) > 1:
                    print(f"ERROR: {next_list}")
                next_el = next_list[0]
                visitados.append(next_el)
                next_list = [j for j, el in enumerate(x[next_el]) if el.x > 0]
            print(visitados)

Depot: 76
[6, 56, 44, 35, 58]
[28, 4, 71]
Depot: 77
[8, 59, 75, 38, 12, 55, 54, 19]
[16]
[17, 50, 39, 3, 66, 13, 7]
[22, 45, 20, 14, 29, 37, 33, 31, 21, 23]
[32, 60, 15, 9, 51, 47]
[61, 68, 46, 26, 64, 1, 74, 2, 24, 57, 49, 48]
[62, 36, 34, 10, 53, 5, 43]
[69, 25, 30]


In [12]:
# caminho dos carros olhando demanda
for k in D:
    print(f"Depot: {k}")
    for i, el in enumerate(u[k]):
        if int(el.x) > 0:
            visitados = [(i, int(el.x))]
            next_list = [(j, int(el.x)) for j, el in enumerate(u[i]) if int(el.x) > 0]
            while len(next_list) > 0:
                if len(next_list) > 1:
                    print(f"ERROR: {next_list}")
                next_el, value = next_list[0]
                visitados.append((next_el, value))
                next_list = [(j, int(el.x)) for j, el in enumerate(u[next_el]) if int(el.x) > 0]
            print([(*el, q[el[0]]) for el in visitados])

Depot: 76
[(6, 10138, 1790), (56, 8348, 1446), (44, 6902, 2610), (35, 2907, 2189), (58, 718, 719)]
[(28, 10138, 3319), (4, 6819, 1107), (71, 1256, 1257)]
Depot: 77
[(8, 10138, 446), (59, 9399, 1202), (75, 8197, 899), (38, 7298, 1518), (12, 5780, 990), (55, 4790, 1574), (54, 3216, 1431), (19, 1785, 1786)]
[(16, 10138, 4506)]
[(17, 10138, 2744), (50, 7394, 1231), (39, 6163, 871), (3, 5292, 2371), (66, 2172, 835), (13, 1337, 765), (7, 572, 573)]
[(22, 10138, 2245), (45, 7893, 1030), (20, 6858, 846), (14, 6012, 641), (29, 5371, 1510), (37, 3861, 1820), (33, 2041, 749), (31, 1292, 546), (21, 746, 307), (23, 439, 440)]
[(32, 10138, 992), (60, 9146, 821), (15, 8325, 4446), (9, 3829, 687), (51, 3142, 1118), (47, 2024, 2025)]
[(61, 10138, 855), (68, 9283, 873), (46, 8410, 924), (26, 7247, 801), (64, 6446, 582), (1, 5864, 797), (74, 5067, 576), (2, 4491, 647), (24, 3844, 1649), (57, 2195, 520), (49, 1675, 416), (48, 1259, 1260)]
[(62, 10138, 2271), (36, 7867, 854), (34, 7013, 1180), (10, 5256, 2

In [13]:
print("Cidades não atendidas")

# cidades não atendidas (olhando arestas)
print(f"aresta:  {([j for j in N if sum([x[i][j].x for i in V]) < 0.5])}")

# nao atendidos olhando demanda
print(f"demanda: {[j for j in N if (q[j] - (sum([u[i][j].x for i in V if i != j]) - sum([u[j][i].x for i in N if i != j]))) > 0.05]}")

# nao atendidos olhando w
print(f"w:       {[j for j in N if w[j].x < 0.5]}")

Cidades não atendidas
aresta:  [0, 11, 18, 27, 40, 41, 42, 52, 63, 65, 67, 70, 72, 73]
demanda: []
w:       [0, 11, 18, 27, 40, 41, 42, 52, 63, 65, 67, 70, 72, 73]


In [14]:
# nao atendidos olhando w
for j in N:
    if w[j].x < 0.5:
        print(f"{j}-> demanda: {q[j]}")

0-> demanda: 0
11-> demanda: 0
18-> demanda: 0
27-> demanda: 0
40-> demanda: 0
41-> demanda: 0
42-> demanda: 0
52-> demanda: 0
63-> demanda: 0
65-> demanda: 0
67-> demanda: 0
70-> demanda: 0
72-> demanda: 0
73-> demanda: 0


In [15]:
# nao atendidos olhando demanda
for j in N:
    enter = (sum([u[i][j].x for i in V if i != j]) - sum([u[j][i].x for i in N if i != j]))
    if (q[j] - enter) > 0.05:
        print(f"{j}-> enter: {enter}  need: {q[j]}")

In [16]:
# nro de veiculos
sum(x[k][i].x for k in D for i in N)

10.0

In [17]:
# cidades que tem mais de dois arcos ligadas a ela (depositos podem ter mais)
print([j for j in V if len([i for i in V if x[j][i].x > 0.5 or x[i][j].x > 0.5]) > 2])

[77]


In [18]:
print(f"distancia media : {sum([c[el] for el in c])/len(c)}")
print(f"distancia maxima: {max([c[el] for el in c])}")
print(f"distancia minima: {min([c[el] for el in c])}")

distancia media : 194.7773214990138
distancia maxima: 538.506
distancia minima: 0.0
